## Tutorial on training a HTS-AT model for audio classification on the ESC-50 Dataset

Referece: 

[HTS-AT: A Hierarchical Token-Semantic Audio Transformer for Sound Classification and Detection, ICASSP 2022](https://arxiv.org/abs/2202.00874)

Following the HTS-AT's paper, in this tutorial, we would show how to use the HST-AT in the training of the ESC-50 Dataset.

The [ESC-50 dataset](https://github.com/karolpiczak/ESC-50) is a labeled collection of 2000 environmental audio recordings suitable for benchmarking methods of environmental sound classification. The dataset consists of 5-second-long recordings organized into 50 semantical classes (with 40 examples per class) loosely arranged into 5 major categories

Before running this tutorial, please make sure that you install the below packages by following steps:

1. download [the codebase](https://github.com/RetroCirce/HTS-Audio-Transformer), and put this tutorial notebook inside the codebase folder.

2. In the github code folder:

    > pip install -r requirements.txt

3. We do not include the installation of PyTorch in the requirment, since different machines require different vereions of CUDA and Toolkits. So make sure you install the PyTorch from [the official guidance](https://pytorch.org/).

4. Install the 'SOX' and the 'ffmpeg', we recommend that you run this code in Linux inside the Conda environment. In that, you can install them by:

    > sudo apt install sox
    
    > conda install -c conda-forge ffmpeg


In [88]:
# import basic packages
import os
import re
import numpy as np
import datetime 
import wget
import sys
import gdown
import zipfile
import librosa
import moviepy.editor as mp
import json
from collections import Counter
# in the notebook, we only can use one GPU
# os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [70]:
# Build the workspace and download the needed files

# def create_path(path):
#     if not os.path.exists(path):
#         os.mkdir(path)

# workspace = "./workspace"
# dataset_path = os.path.join(workspace, "esc-50")
# checkpoint_path = os.path.join(workspace, "ckpt")
# esc_raw_path = os.path.join(dataset_path, 'raw')


# create_path(workspace)
# create_path(dataset_path)
# create_path(checkpoint_path)
# create_path(esc_raw_path)


# # download the esc-50 dataset

# if not os.path.exists(os.path.join(dataset_path, 'ESC-50-master.zip')):
#     print("-------------Downloading ESC-50 Dataset-------------")
#     wget.download('https://github.com/karoldvl/ESC-50/archive/master.zip', out=dataset_path)
#     with zipfile.ZipFile(os.path.join(dataset_path, 'ESC-50-master.zip'), 'r') as zip_ref:
#         zip_ref.extractall(esc_raw_path)
#     print("-------------Success-------------")

# if not os.path.exists(os.path.join(checkpoint_path,'htsat_audioset_pretrain.ckpt')):
#     gdown.download(id='1OK8a5XuMVLyeVKF117L8pfxeZYdfSDZv', output=os.path.join(checkpoint_path,'htsat_audioset_pretrain.ckpt'))




In [71]:
# Process ESC-50 Dataset
# meta_path = os.path.join(esc_raw_path, 'ESC-50-master', 'meta', 'esc50.csv')
# audio_path = os.path.join(esc_raw_path, 'ESC-50-master', 'audio')
# resample_path = os.path.join(dataset_path, 'resample')
# savedata_path = os.path.join(dataset_path, 'esc-50-data.npy')
# create_path(resample_path)

# meta = np.loadtxt(meta_path , delimiter=',', dtype='str', skiprows=1)
# audio_list = os.listdir(audio_path)

# # resample
# print("-------------Resample ESC-50-------------")
# for f in audio_list:
#     full_f = os.path.join(audio_path, f)
#     resample_f = os.path.join(resample_path, f)
#     if not os.path.exists(resample_f):
#         os.system('sox -V1 ' + full_f + ' -r 32000 ' + resample_f)
# print("-------------Success-------------")

# print("-------------Build Dataset-------------")
# output_dict = [[] for _ in range(5)]
# for label in meta:
#     name = label[0]
#     fold = label[1]
#     target = label[2]
#     y, sr = librosa.load(os.path.join(resample_path, name), sr = None)
#     output_dict[int(fold) - 1].append(
#         {
#             "name": name,
#             "target": int(target),
#             "waveform": y
#         }
#     )
# np.save(savedata_path, output_dict)
# print("-------------Success-------------")
    

In [89]:
# Load the model package
import torch
from torch.utils.data import DataLoader
from torch.utils.data.distributed import DistributedSampler
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
import warnings

from utils import create_folder, dump_config, process_idc
import config 
from sed_model import SEDWrapper, Ensemble_SEDWrapper
from data_generator import ESC_Dataset
from model.htsat import HTSAT_Swin_Transformer



In [90]:
# Data Preparation
class data_prep(pl.LightningDataModule):
    def __init__(self, train_dataset, eval_dataset, device_num):
        super().__init__()
        self.train_dataset = train_dataset
        self.eval_dataset = eval_dataset
        self.device_num = device_num

    def train_dataloader(self):
        train_sampler = DistributedSampler(self.train_dataset, shuffle = False) if self.device_num > 1 else None
        train_loader = DataLoader(
            dataset = self.train_dataset,
            num_workers = config.num_workers,
            batch_size = config.batch_size // self.device_num,
            shuffle = False,
            sampler = train_sampler
        )
        return train_loader
    def val_dataloader(self):
        eval_sampler = DistributedSampler(self.eval_dataset, shuffle = False) if self.device_num > 1 else None
        eval_loader = DataLoader(
            dataset = self.eval_dataset,
            num_workers = config.num_workers,
            batch_size = config.batch_size // self.device_num,
            shuffle = False,
            sampler = eval_sampler
        )
        return eval_loader
    def test_dataloader(self):
        test_sampler = DistributedSampler(self.eval_dataset, shuffle = False) if self.device_num > 1 else None
        test_loader = DataLoader(
            dataset = self.eval_dataset,
            num_workers = config.num_workers,
            batch_size = config.batch_size // self.device_num,
            shuffle = False,
            sampler = test_sampler
        )
        return test_loader
    

In [74]:
# Set the workspace
# device_num = torch.cuda.device_count()
# print("each batch size:", config.batch_size // device_num)

# full_dataset = np.load(os.path.join(config.dataset_path, "esc-50-data.npy"), allow_pickle = True)

# # set exp folder
# exp_dir = os.path.join(config.workspace, "results", config.exp_name)
# checkpoint_dir = os.path.join(config.workspace, "results", config.exp_name, "checkpoint")
# if not config.debug:
#     create_folder(os.path.join(config.workspace, "results"))
#     create_folder(exp_dir)
#     create_folder(checkpoint_dir)
#     dump_config(config, os.path.join(exp_dir, config.exp_name), False)

# print("Using ESC")
# dataset = ESC_Dataset(
#     dataset = full_dataset,
#     config = config,
#     eval_mode = False
# )
# eval_dataset = ESC_Dataset(
#     dataset = full_dataset,
#     config = config,
#     eval_mode = True
# )

# audioset_data = data_prep(dataset, eval_dataset, device_num)
# checkpoint_callback = ModelCheckpoint(
#     monitor = "acc",
#     filename='l-{epoch:d}-{acc:.3f}',
#     save_top_k = 20,
#     mode = "max"
# )




In [91]:
config=config

In [92]:
# Set the Trainer
# trainer = pl.Trainer(
#     deterministic=True,
#     default_root_dir = checkpoint_dir,
#     gpus = device_num, 
#     val_check_interval = 1.0,
#     max_epochs = config.max_epoch,
#     auto_lr_find = True,    
#     sync_batchnorm = True,
#     callbacks = [checkpoint_callback],
#     accelerator = "ddp" if device_num > 1 else None,
#     num_sanity_val_steps = 0,
#     resume_from_checkpoint = None, 
#     replace_sampler_ddp = False,
#     gradient_clip_val=1.0
# )

sed_model = HTSAT_Swin_Transformer(
    spec_size=config.htsat_spec_size,
    patch_size=config.htsat_patch_size,
    in_chans=1,
    num_classes=config.classes_num,
    window_size=config.htsat_window_size,
    config = config,
    depths = config.htsat_depth,
    embed_dim = config.htsat_dim,
    patch_stride=config.htsat_stride,
    num_heads=config.htsat_num_head
)

model = SEDWrapper(
    sed_model = sed_model, 
    config = config,
    dataset = ''
)

if config.resume_checkpoint is not None:
    print("Load Checkpoint from ", config.resume_checkpoint)
    ckpt = torch.load(config.resume_checkpoint, map_location="cpu")
    ckpt["state_dict"].pop("sed_model.head.weight")
    ckpt["state_dict"].pop("sed_model.head.bias")
    # finetune on the esc and spv2 dataset
    # ckpt["state_dict"].pop("sed_model.tscam_conv.weight")
    # ckpt["state_dict"].pop("sed_model.tscam_conv.bias")
    # model.load_state_dict(ckpt["state_dict"], strict=False)



In [77]:
# # Training the model
# # You can set different fold index by setting 'esc_fold' to any number from 0-4 in esc_config.py
# trainer.fit(model, audioset_data)

In [93]:
data = np.load('', allow_pickle=True)

## Now Let us Check the Result

Find the path of your saved checkpoint and paste it in the below variable.
Then you are able to follow the below code for checking the prediction result of any sample you like.

In [94]:
# infer the single data to check the result
# get a model you saved
model_path = ''

# get the groundtruth
# meta = np.loadtxt(meta_path , delimiter=',', dtype='str', skiprows=1)
# gd = {}
# for label in meta:
#     name = label[0]
#     target = label[2]
#     gd[name] = target

class Audio_Classification:
    def __init__(self, model_path, config):
        super().__init__()

        self.device = torch.device('cpu')
        self.sed_model = HTSAT_Swin_Transformer(
            spec_size=config.htsat_spec_size,
            patch_size=config.htsat_patch_size,
            in_chans=1,
            num_classes=config.classes_num,
            window_size=config.htsat_window_size,
            config = config,
            depths = config.htsat_depth,
            embed_dim = config.htsat_dim,
            patch_stride=config.htsat_stride,
            num_heads=config.htsat_num_head
        )
        ckpt = torch.load(model_path, map_location="cpu")
        temp_ckpt = {}
        for key in ckpt["state_dict"]:
            temp_ckpt[key[10:]] = ckpt['state_dict'][key]
        self.sed_model.load_state_dict(temp_ckpt)
        self.sed_model.to(self.device)
        self.sed_model.eval()


    def predict(self, audiofile):

        if audiofile:
            waveform, sr = librosa.load(audiofile, sr=32000)

            with torch.no_grad():
                x = torch.from_numpy(waveform).float().to(self.device)
                output_dict = self.sed_model(x[None, :], None, True)
                # print(output_dict)
                pred = output_dict['clipwise_output']
                pred_post = pred[0].detach().cpu().numpy()
                pred_label = np.argmax(pred_post)
                pred_prob = np.max(pred_post)
                if pred_label == 0:
                    pred_post = np.delete(pred_post, 0)
                    pred_label = np.argmax(pred_post)
                    pred_prob = np.max(pred_post)
                    pred_label = np.argmax(np.delete(pred_post, 0))+1
                    pred_prob = np.max(np.delete(pred_post, 0))
                if pred_label == 431:
                    pred_post = np.delete(pred_post, 431)
                    pred_label = np.argmax(pred_post)
                    pred_prob = np.max(pred_post)
                    pred_label = np.argmax(np.delete(pred_post, 431))+1
                    pred_prob = np.max(np.delete(pred_post, 431))
                    
            return pred_label, pred_prob


In [80]:
data[427]

array(['Gunshot, gunfire', list([426]), list([428, 429, 430, 431])],
      dtype=object)

In [97]:
classes = ["Emergency vehicle", "Gunshot, gunfire", "Helicopter", "Fireworks", "Explosion", 
           "Civil defense siren", "Cough", "Motorcycle", "Race car, auto racing", 
           "Crying, sobbing", "Fire alarm", 'Vehicle', "Car", "Motor vehicle (road)",
           "Skidding", "Battle cry", "Accelerating, revving, vroom", "Police car (siren)",
           "Fire engine, fire truck (siren)", "Conversation", "Artillery fire", "Alarm",
           "Machine gun", "Crowd", "Ambulance (siren)", "Firecracker", "Whip", "Toot" "Snort",
           "Cap gun", "Fusillade", "Yell"]

n=3
my_clip = mp.VideoFileClip('video/myanmar.mp4')
remainder = int(my_clip.duration % n)
int_seg = int(my_clip.duration - remainder)
durations = list(range(0, int_seg+n, n))
paired_durations = [[x, y] for x, y in zip(durations, durations[1:])]
file_name = my_clip.filename.split('/')[1].split('.')[0]
# file_name = file_name.replace('_', '')
print(file_name)


myanmar


In [83]:
os.makedirs(f'video/{file_name}', exist_ok=True)

# creates audio clips
for i in paired_durations:
    clips = my_clip.audio.subclip(t_start=i[0], t_end=i[1])
    clips.write_audiofile(f'video/{file_name}/{file_name}_{i[0]}_{i[1]}.wav', logger=None)

# clips remainder of video
if remainder > 0:
    remainder_clip = my_clip.audio.subclip(t_start=paired_durations[-1][1], t_end=paired_durations[-1][1]+remainder)
    remainder_clip.write_audiofile(f'video/{file_name}/{file_name}_{paired_durations[-1][1]}_{paired_durations[-1][1]+remainder}.wav', logger=None)

In [ ]:
def extract_integer(filename):
    return int(filename.split('.')[0].split('_')[1])

audio_list = sorted(os.listdir(f'video/{file_name}'), key=extract_integer)


In [85]:
filtered_classes = {
'Gunshot/Boom': {"Gunshot, gunfire", "Fireworks", "Explosion", "Artillery fire", "Machine gun", "Firecracker", "Whip", "Cap gun", "Fusillade"},
'Siren/Alarm': {"Fire alarm", "Emergency vehicle", "Civil defense siren", "Police car (siren)", "Fire engine, fire truck (siren)", "Alarm", "Ambulance (siren)"},
'Coughing/Gasping': {"Snort", "Cough"},
'Crying': {"Crying, sobbing"},
'Helicopter': {"Helicopter"},
'Conversation': {"Conversation"},
'Crowd Noise': {"Crowd"},
'Yelling': {"Battle cry", "Yell"},
'Horn': {"Toot"},
'Vehicle Skidding': {"Skidding"},
'Vehicle Noise': {"Motorcycle", "Race car, auto racing", "Vehicle", "Car", "Motor vehicle (road)", "Accelerating, revving, vroom"}
}

In [86]:
def GetKey(val):
    for key, value in filtered_classes.items():
        if val in value:
            return key

In [ ]:
# performs audio classification on clips in directory
results_dict = {}
unfiltered = []
# performs audio classification on clips in directory
Audiocls = Audio_Classification(model_path, config)
for clip in audio_list:
    pred_label, pred_prob = Audiocls.predict(f'video/{file_name}/{clip}')
    unfiltered.append((clip, data[pred_label][0]))

In [ ]:
filtered = []
# converts class detections to per second 
for entry in unfiltered:
    slice_ts = entry[0].split('.')[0].split('_')[1:]
    for sec in range(int(slice_ts[0]), int(slice_ts[-1])):
        filtered.append((sec+1, entry[1]))

In [ ]:
# filters to only classes in filtered_classes
audioData = [(i[0], GetKey(i[1])) for i in filtered if i[1] in classes]
# Counts # of detections per class 
totals = dict(Counter([i[1] for i in audioData]))

In [ ]:
# compiles and writes to dictionary
results_dict['uniqueId'] = my_clip.filename.split('video/')[1]
results_dict['totals'] = totals
results_dict['audioData'] = audioData
results_dict['audioGraph'] = []

In [ ]:
results_dict

{'uniqueId': 'myanmar.mp4',
 'totals': {'Gunshot/Boom': 12, 'Crowd Noise': 3},
 'audioData': [(64, 'Gunshot/Boom'),
  (65, 'Gunshot/Boom'),
  (66, 'Gunshot/Boom'),
  (70, 'Gunshot/Boom'),
  (71, 'Gunshot/Boom'),
  (72, 'Gunshot/Boom'),
  (76, 'Gunshot/Boom'),
  (77, 'Gunshot/Boom'),
  (78, 'Gunshot/Boom'),
  (88, 'Gunshot/Boom'),
  (89, 'Gunshot/Boom'),
  (90, 'Gunshot/Boom'),
  (115, 'Crowd Noise'),
  (116, 'Crowd Noise'),
  (117, 'Crowd Noise')],
 'audioGraph': []}